### Analyze the stiffnesses calculated from all files
This notebook removes outliers from the calculated stiffnesses and calculates the average stiffnesses for each construct and condition as reported in the paper. Then it calculates the stiffness of PCDH15 alone (accounting for the anchors in series with the protein). This notebook also propagates the errors.

Camila Villasante <br>
10-18-2023

In [ ]:
# Import modules
import pickle
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
# Import pickle file generated from CalculateEnthalpicStiffnesses_20231018.py
file_to_read = open(PATH TO FILE, "rb")
analysis_output = pickle.load(file_to_read)

In [ ]:
# Assign the stiffness values to different lists based on construct and condition

#V507D
V507DDimer_3 = analysis_output['V507DDimer_3']
V507DDimer_20 = analysis_output['V507DDimer_20']
V507DDimer_0 = analysis_output['V507DDimer_0']

# Wild type
Dimer_3 = analysis_output['Dimer_3']
Dimer_20 = analysis_output['Dimer_20']
Dimer_0 = analysis_output['Dimer_0']

# Anchors
Anchors_3 = analysis_output['Anchors']

In [ ]:
# Flatten lists because they are lists of lists right now
V507DDimer_3 = [i for sublist in V507DDimer_3 for i in sublist]
V507DDimer_20 = [i for sublist in V507DDimer_20 for i in sublist]
V507DDimer_0 = [i for sublist in V507DDimer_0 for i in sublist]

Dimer_3 = [i for sublist in Dimer_3 for i in sublist]
Dimer_20 = [i for sublist in Dimer_20 for i in sublist]
Dimer_0 = [i for sublist in Dimer_0 for i in sublist]

Anchors_3 = [i for sublist in Anchors_3 for i in sublist]


In [ ]:
# Make dataframe of all stiffnesses to make processing easier
stiffnesses_df = pd.DataFrame({'V507DDimer_3':pd.Series(V507DDimer_3),'V507DDimer_20':pd.Series(V507DDimer_20),'V507DDimer_0':pd.Series(V507DDimer_0),'Dimer_3':pd.Series(Dimer_3),'Dimer_20':pd.Series(Dimer_20),'Dimer_0':pd.Series(Dimer_0), 'Anchors_3':pd.Series(Anchors_3)})


In [ ]:
# Remove outliers

# Inititate lists to add values to
stiffnesses_wo_outliers = []
sems=[] # SEMs 
stds=[] # Standard deviations
for col in stiffnesses_df: 
    print('There are',stiffnesses_df[col].notna().sum(),'values in', col)
    print('Mean is',np.nanmean(stiffnesses_df[col]))
    
    # Calculate IQR
    stiffnesses_Q1 = np.percentile(stiffnesses_df[col], 25, interpolation = 'midpoint')
    stiffnesses_Q3 = np.percentile(stiffnesses_df[col], 75, interpolation = 'midpoint')
    stiffnesses_IQR = stiffnesses_Q3 - stiffnesses_Q1
    
    # Upper bound of values to keep is third quartile + 1.5*IQR
    stiffnesses_upper = np.where(stiffnesses_df[col] >= (stiffnesses_Q3+1.5*stiffnesses_IQR))
    # Lower bound of values to keep is first quartile - 1.5*IQR
    stiffnesses_lower = np.where(stiffnesses_df[col] <= (stiffnesses_Q1-1.5*stiffnesses_IQR))

    # Drop outliers outside of upper and lower bounds
    stiffnesses_df.drop(dydx_upper[0], inplace = True)
    stiffnesses_df.drop(dydx_lower[0], inplace = True)

    # Mean without outliers
    avg_stiffness = np.nanmean(stiffness_df[col])
    stiffnesses_wo_outliers.append(avg_stiffness)
    sem = stats.sem(stiffness_df[col].dropna())
    sems.append(sem)
    std = np.std(stiffness_df[col].dropna())
    stds.append(std)
    print('Mean without outliers is',avg_stiffnesses,'+/-', sem)


In [ ]:
# Calculate stiffness of PCDH15 alone (need to account for the anchors in series)
# Treat PCDH15 and the anchors as two springs in series

# Define stiffnesses for each construct and condition from output in previous cell
k_tot_V507DDimer_3 = stiffnesses_wo_outliers[0]
k_tot_V507DDimer_20 = stiffnesses_wo_outliers[1]
k_tot_V507DDimer_0 = stiffnesses_wo_outliers[2]
k_tot_Dimer_3 = stiffnesses_wo_outliers[3]
k_tot_Dimer_20 = stiffnesses_wo_outliers[4]
k_tot_Dimer_0 = stiffnesses_wo_outliers[5]
k_anchors = stiffnesses_wo_outliers[6]

# Calculate stiffness of PCDH15 alone for each construct and condition
k_V507DDimer_3 = (k_tot_V507DDimer_3*k_anchors)/(k_anchors - k_tot_V507DDimer_3)
k_V507DDimer_20 = (k_tot_V507DDimer_20*k_anchors)/(k_anchors - k_tot_V507DDimer_20)
k_V507DDimer_0 = (k_tot_V507DDimer_0*k_anchors)/(k_anchors - k_tot_V507DDimer_0)
k_Dimer_3 = (k_tot_Dimer_3*k_anchors)/(k_anchors - k_tot_Dimer_3)
k_Dimer_20 = (k_tot_Dimer_20*k_anchors)/(k_anchors - k_tot_Dimer_20)
k_Dimer_0 = (k_tot_Dimer_0*k_anchors)/(k_anchors - k_tot_Dimer_0)


### Error propagation

In [ ]:
# Define SEMs for each construct and condition from calculation above
sem_V507DDimer_3 = sems[0]
sem_V507DDimer_20 = sems[1]
sem_V507DDimer_0 = sems[2]

sem_Dimer_3 = sems[3]
sem_Dimer_20 = sems[4]
sem_Dimer_0 = sems[5]

sem_anchors_3 = sems[6]

In [ ]:
## Equations for error propgation
# V507D, 3 mM Ca2+
numerator_V507DDimer_3 = k_tot_V507DDimer_3*k_anchors
denominator_V507DDimer_3 = k_anchors-k_tot_V507DDimer_3

error_numerator_V507DDimer_3 = numerator_V507DDimer_3*np.sqrt((sem_toypeptide_3/k_anchors)**2+(sem_V507DDimer_3/k_tot_V507DDimer_3)**2)
error_denominator_V507DDimer_3 = np.sqrt((sem_toypeptide_3)**2+(sem_V507DDimer_3)**2)
error_total_V507DDimer_3 = (k_V507DDimer_3)*np.sqrt((error_denominator_V507DDimer_3/denominator_V507DDimer_3)**2+(error_numerator_V507DDimer_3/(numerator_V507DDimer_3))**2)

# V507D, 20 uM Ca2+
numerator_V507DDimer_20 = k_tot_V507DDimer_20*k_anchors
denominator_V507DDimer_20 = k_anchors-k_tot_V507DDimer_20

error_numerator_V507DDimer_20 = numerator_V507DDimer_20*np.sqrt((sem_toypeptide_3/k_anchors)**2+(sem_V507DDimer_20/k_tot_V507DDimer_20)**2)
error_denominator_V507DDimer_20 = np.sqrt((sem_toypeptide_3)**2+(sem_V507DDimer_20)**2)
error_total_V507DDimer_20 = (k_V507DDimer_20)*np.sqrt((error_denominator_V507DDimer_20/denominator_V507DDimer_20)**2+(error_numerator_V507DDimer_20/(numerator_V507DDimer_20))**2)

# V507D, 0 M Ca2+ 1 mM EDTA
numerator_V507DDimer_0 = k_tot_V507DDimer_0*k_anchors
denominator_V507DDimer_0 = k_anchors-k_tot_V507DDimer_0

error_numerator_V507DDimer_0 = numerator_V507DDimer_0*np.sqrt((sem_toypeptide_3/k_anchors)**2+(sem_V507DDimer_0/k_tot_V507DDimer_0)**2)
error_denominator_V507DDimer_0 = np.sqrt((sem_toypeptide_3)**2+(sem_V507DDimer_0)**2)
error_total_V507DDimer_0 = (k_V507DDimer_0)*np.sqrt((error_denominator_V507DDimer_0/denominator_V507DDimer_0)**2+(error_numerator_V507DDimer_0/(numerator_V507DDimer_0))**2)


In [ ]:
## Equations for error propgation
# Wild type, 3 mM Ca2+
numerator_Dimer_3 = k_tot_Dimer_3*k_anchors
denominator_Dimer_3 = k_anchors-k_tot_Dimer_3

error_numerator_Dimer_3 = numerator_Dimer_3*np.sqrt((sem_toypeptide_3/k_anchors)**2+(sem_Dimer_3/k_tot_Dimer_3)**2)
error_denominator_Dimer_3 = np.sqrt((sem_toypeptide_3)**2+(sem_Dimer_3)**2)
error_total_Dimer_3 = (k_Dimer_3)*np.sqrt((error_denominator_Dimer_3/denominator_Dimer_3)**2+(error_numerator_Dimer_3/(numerator_Dimer_3))**2)

# Wild type, 20 uM Ca2+
numerator_Dimer_20 = k_tot_Dimer_20*k_anchors
denominator_Dimer_20 = k_anchors-k_tot_Dimer_20

error_numerator_Dimer_20 = numerator_Dimer_20*np.sqrt((sem_toypeptide_3/k_anchors)**2+(sem_Dimer_20/k_tot_Dimer_20)**2)
error_denominator_Dimer_20 = np.sqrt((sem_toypeptide_3)**2+(sem_Dimer_20)**2)
error_total_Dimer_20 = (k_Dimer_20)*np.sqrt((error_denominator_Dimer_20/denominator_Dimer_20)**2+(error_numerator_Dimer_20/(numerator_Dimer_20))**2)

# Wild type, 0 M Ca2+ 1 mM EDTA
numerator_Dimer_0 = k_tot_Dimer_0*k_anchors
denominator_Dimer_0 = k_anchors-k_tot_Dimer_0

error_numerator_Dimer_0 = numerator_Dimer_0*np.sqrt((sem_toypeptide_3/k_anchors)**2+(sem_Dimer_0/k_tot_Dimer_0)**2)
error_denominator_Dimer_0 = np.sqrt((sem_toypeptide_3)**2+(sem_Dimer_0)**2)
error_total_Dimer_0 = (k_Dimer_0)*np.sqrt((error_denominator_Dimer_0/denominator_Dimer_0)**2+(error_numerator_Dimer_0/(numerator_Dimer_0))**2)
